##### Why passing argument `date_format=None` in function?
- How to **cast** multiple columns to specific data types?
- How the table looks before and after casting?
- when the type is **"date" or "timestamp"**, you can **optionally** pass a **date_format**.
- so that Spark knows how to parse the **string** values into proper **DateType or TimestampType**.

In [0]:
import pyspark.sql.functions as F

In [0]:
# All columns are in string type
# All rows of date & timestamp are in default format

# Sample data
data = [
    ("Albert", "25", "55000.50", "true", "2025-08-25", "6", "2025-08-25T15:30:00.000+00:00"),      # All rows of date & timestamp are in default format
    ("Baskar", "30", "72000.00", "false", "2024-12-31", "7", "2024-12-31T08:45:15.000+00:00"),
    ("Chetan", "27", "63000.75", "true", "2023-01-15", "8", "2023-01-15T20:00:00.000+00:00"),
    ("Dravid", "26", "66000.50", "true", "2024-06-20", "9", "2024-05-22T15:40:55.000+00:00"),
    ("Nishant", "32", "98700.00", "false", "2023-10-21", "10", "2020-10-31T06:45:45.000+00:00"),
    ("David", "29", "34512.75", "true", "2023-03-15", "4", "2021-08-28T20:15:55.000+00:00"),
    ("Mohan", "33", "34908.50", "true", "2022-04-15", "12", "2019-09-29T19:35:55.000+00:00"),
    ("Niroop", "35", "49654.98", "false", "2021-02-18", "3", "2024-05-22T08:45:25.000+00:00"),
    ("Pushpa", "23", "44111.99", "true", "2020-07-19", "5", "2023-09-25T20:00:00.000+00:00")
]

# Define schema
columns = ["name", "age", "salary", "is_active", "join_date", "Exp" ,"last_login"]

df_default_format = spark.createDataFrame(data, columns)
display(df_default_format)

name,age,salary,is_active,join_date,Exp,last_login
Albert,25,55000.50,true,2025-08-25,6,2025-08-25T15:30:00.000+00:00
Baskar,30,72000.00,false,2024-12-31,7,2024-12-31T08:45:15.000+00:00
Chetan,27,63000.75,true,2023-01-15,8,2023-01-15T20:00:00.000+00:00
Dravid,26,66000.50,true,2024-06-20,9,2024-05-22T15:40:55.000+00:00
Nishant,32,98700.00,false,2023-10-21,10,2020-10-31T06:45:45.000+00:00
David,29,34512.75,true,2023-03-15,4,2021-08-28T20:15:55.000+00:00
Mohan,33,34908.50,true,2022-04-15,12,2019-09-29T19:35:55.000+00:00
Niroop,35,49654.98,false,2021-02-18,3,2024-05-22T08:45:25.000+00:00
Pushpa,23,44111.99,true,2020-07-19,5,2023-09-25T20:00:00.000+00:00


In [0]:
col_type_map = {
    "age": "int",
    "salary": "double",
    "is_active": "boolean",
    "join_date": "date",
    "Exp": "Int",
    "last_login": "timestamp"
}

##### Scenario 01
- **date_format=None**
- **Default argument behavior**

In [0]:
def cast_dataframe_columns_wo(df, col_type_map, date_format=None):
    for col_name, data_type in col_type_map.items():
        data_type = data_type.lower()

        if data_type == "string":
            df = df.withColumn(col_name, F.col(col_name).cast("string"))

        elif data_type in ["int", "Int", "integer"]:
            df = df.withColumn(col_name, F.col(col_name).cast("int"))

        elif data_type in ["double", "float"]:
            df = df.withColumn(col_name, F.col(col_name).cast("double"))

        elif data_type == "boolean":
            df = df.withColumn(col_name, F.col(col_name).cast("boolean"))

        elif data_type == "date":
            df = df.withColumn(col_name, F.to_date(F.col(col_name)))              # since date & timestamp are in default format, no need to pass date_format

        elif data_type == "timestamp":
            df = df.withColumn(col_name, F.to_timestamp(F.col(col_name)))         # since date & timestamp are in default format, no need to pass date_format
    return df

In [0]:
# Apply casting with correct date / timestamp format
# df_default_format: date & timestamp are in default format
# since date & timestamp are in default format, no need to pass date_format -> cast_dataframe_columns_wo(df, col_type_map, date_format=None)

df_casted_wo = cast_dataframe_columns_wo(df_default_format, col_type_map)
display(df_casted_wo)

name,age,salary,is_active,join_date,Exp,last_login
Albert,25,55000.5,true,2025-08-25,6,2025-08-25T15:30:00.000Z
Baskar,30,72000.0,false,2024-12-31,7,2024-12-31T08:45:15.000Z
Chetan,27,63000.75,true,2023-01-15,8,2023-01-15T20:00:00.000Z
Dravid,26,66000.5,true,2024-06-20,9,2024-05-22T15:40:55.000Z
Nishant,32,98700.0,false,2023-10-21,10,2020-10-31T06:45:45.000Z
David,29,34512.75,true,2023-03-15,4,2021-08-28T20:15:55.000Z
Mohan,33,34908.5,true,2022-04-15,12,2019-09-29T19:35:55.000Z
Niroop,35,49654.98,false,2021-02-18,3,2024-05-22T08:45:25.000Z
Pushpa,23,44111.99,true,2020-07-19,5,2023-09-25T20:00:00.000Z


| Output schema BEFORE casting |   Output schema AFTER casting |
|------------------------------|-------------------------------|
|         `name`:string        |         `name`:string         |  
|         `age`:`string`       |         `age`:`integer`       |
|        `salary`:`string`     |        `salary`:`double`      |
|       `is_active`:`string`   |      `is_active`:`boolean`    |
|       `join_date`:`string`   |        `join_date`:`date`     |
|         `Exp`:`string`       |       `Exp`:`integer`         |
|       `last_login`:`string`  |     `last_login`:`timestamp`  |    

In [0]:
# Apply casting with correct date / timestamp format
# df_cust_format: date & timestamp are in custom format
# since date & timestamp are in custom format, pass date_format -> cast_dataframe_columns_wo(df, col_type_map, date_format="%m-%d-%Y")
# date_format="%m-%d-%Y" -> month-day-year
# date_format="%m-%d-%Y %H:%M:%S.%f%z" -> month-day-year hour:minute:second.microsecond
# since date_format not passed getting error -> [CAST_INVALID_INPUT] The value '08-25-2025' of the type "STRING" cannot be cast to "DATE" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018

df_casted_cust = cast_dataframe_columns_wo(df_cust_format, col_type_map)
display(df_casted_cust)

---------------------------------------------------------------------------
DateTimeException                         Traceback (most recent call last)
File <command-2022302555744395>, line 9
      1 # Apply casting with correct date / timestamp format
      2 # df_cust_format: date & timestamp are in custom format
      3 # since date & timestamp are in custom format, pass date_format -> cast_dataframe_columns_wo(df, col_type_map, date_format="%m-%d-%Y")
      4 # date_format="%m-%d-%Y" -> month-day-year
      5 # date_format="%m-%d-%Y %H:%M:%S.%f%z" -> month-day-year hour:minute:second.microsecond
      6 # since date_format not passed getting error -> [CAST_INVALID_INPUT] The value '08-25-2025' of the type "STRING" cannot be cast to "DATE" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018
      8 df_casted_cust = cast_dataframe_columns_wo(df_cust_format, col_typ

**date_format** has a **default value** of **None**.

      def cast_dataframe_columns_wo(df, col_type_map, date_format=None):

- If the user **does not** provide a **date_format**, the function **still works**, it will just treat **date_format** as **None**.
   - Spark uses its **default date** `parsing rules`, which expect the input to be in the **standard format**  

         # No format passed → date_format is None
         cast_dataframe_columns_wo(df, {"join_date": "date"})

         -> yyyy-MM-dd (for date)
         -> yyyy-MM-dd HH:mm:ss (for timestamp)

##### Scenario 02
- date = "MM-dd-yyyy" -> custom format
- date_format = "yyyy-MM-dd'T'HH:mm:ss.SSSXXX"

In [0]:
# All columns are in string type
# All rows of date & timestamp are in custom format

# Sample data
data = [
    ("Albert", "25", "55000.50", "true", "08-25-2025", "6", "08-25-2025T15:30:00.000+00:00"),     # All rows of date & timestamp are in default format
    ("Baskar", "30", "72000.00", "false", "12-30-2024", "7", "12-31-2024T08:45:15.000+00:00"),
    ("Chetan", "27", "63000.75", "true", "01-15-2023", "8", "01-15-2023T20:00:00.000+00:00"),
    ("Dravid", "26", "66000.50", "true", "06-20-2024", "9", "05-22-2024T15:40:55.000+00:00"),
    ("Nishant", "32", "98700.00", "false", "10-21-2023", "10", "10-31-2020T06:45:45.000+00:00"),
    ("David", "29", "34512.75", "true", "03-15-2023", "4", "08-28-2021T20:15:55.000+00:00"),
    ("Mohan", "33", "34908.50", "true", "04-15-2022", "12", "09-29-2019T19:35:55.000+00:00"),
    ("Niroop", "35", "49654.98", "false", "02-18-2021", "3", "05-22-2024T08:45:25.000+00:00"),
    ("Pushpa", "23", "44111.99", "true", "07-19-2020", "5", "09-25-2023T20:00:00.000+00:00")
]

# Define schema
columns = ["name", "age", "salary", "is_active", "join_date", "Exp" ,"last_login"]

df_cust_format = spark.createDataFrame(data, columns)
display(df_cust_format)

name,age,salary,is_active,join_date,Exp,last_login
Albert,25,55000.50,true,08-25-2025,6,08-25-2025T15:30:00.000+00:00
Baskar,30,72000.00,false,12-30-2024,7,12-31-2024T08:45:15.000+00:00
Chetan,27,63000.75,true,01-15-2023,8,01-15-2023T20:00:00.000+00:00
Dravid,26,66000.50,true,06-20-2024,9,05-22-2024T15:40:55.000+00:00
Nishant,32,98700.00,false,10-21-2023,10,10-31-2020T06:45:45.000+00:00
David,29,34512.75,true,03-15-2023,4,08-28-2021T20:15:55.000+00:00
Mohan,33,34908.50,true,04-15-2022,12,09-29-2019T19:35:55.000+00:00
Niroop,35,49654.98,false,02-18-2021,3,05-22-2024T08:45:25.000+00:00
Pushpa,23,44111.99,true,07-19-2020,5,09-25-2023T20:00:00.000+00:00


In [0]:
def cast_dataframe_columns_tformat(df, col_type_map, date_format=None):
    for col_name, data_type in col_type_map.items():
        data_type = data_type.lower()

        if data_type == "string":
            df = df.withColumn(col_name, F.col(col_name).cast("string"))

        elif data_type in ["int", "Int", "integer"]:
            df = df.withColumn(col_name, F.col(col_name).cast("int"))

        elif data_type in ["double", "float"]:
            df = df.withColumn(col_name, F.col(col_name).cast("double"))

        elif data_type == "boolean":
            df = df.withColumn(col_name, F.col(col_name).cast("boolean"))

        elif data_type == "date":
            # Handle custom date format "08-25-2025" -> "MM-dd-yyyy"
            df = df.withColumn(col_name, F.to_date(F.col(col_name), "MM-dd-yyyy"))

        elif data_type == "timestamp":
            df = df.withColumn(col_name, F.to_timestamp(F.col(col_name), date_format))
    return df

In [0]:
# Apply casting with correct date / timestamp format
df_casted_tformat = cast_dataframe_columns_tformat(df_cust_format, col_type_map, "MM-dd-yyyy'T'HH:mm:ss.SSSXXX")
display(df_casted_tformat)

name,age,salary,is_active,join_date,Exp,last_login
Albert,25,55000.5,true,2025-08-25,6,2025-08-25T15:30:00.000Z
Baskar,30,72000.0,false,2024-12-30,7,2024-12-31T08:45:15.000Z
Chetan,27,63000.75,true,2023-01-15,8,2023-01-15T20:00:00.000Z
Dravid,26,66000.5,true,2024-06-20,9,2024-05-22T15:40:55.000Z
Nishant,32,98700.0,false,2023-10-21,10,2020-10-31T06:45:45.000Z
David,29,34512.75,true,2023-03-15,4,2021-08-28T20:15:55.000Z
Mohan,33,34908.5,true,2022-04-15,12,2019-09-29T19:35:55.000Z
Niroop,35,49654.98,false,2021-02-18,3,2024-05-22T08:45:25.000Z
Pushpa,23,44111.99,true,2020-07-19,5,2023-09-25T20:00:00.000Z


##### Scenario 03
- date_format_map=**None**

In [0]:
def cast_dataframe_columns_map_wo(df, col_type_map, date_format_map=None):
    for col_name, data_type in col_type_map.items():
        data_type = data_type.lower()

        if data_type == "string":
            df = df.withColumn(col_name, F.col(col_name).cast("string"))

        elif data_type in ["int", "Int", "integer"]:
            df = df.withColumn(col_name, F.col(col_name).cast("int"))

        elif data_type in ["double", "float"]:
            df = df.withColumn(col_name, F.col(col_name).cast("double"))

        elif data_type == "boolean":
            df = df.withColumn(col_name, F.col(col_name).cast("boolean"))

        elif data_type == "date":
            df = df.withColumn(col_name, F.to_date(F.col(col_name)))

        elif data_type == "timestamp":
            df = df.withColumn(col_name, F.to_timestamp(F.col(col_name)))
    return df

In [0]:
df_casted_map_wo = cast_dataframe_columns_map_wo(df_default_format, col_type_map)
display(df_casted_map_wo)

name,age,salary,is_active,join_date,Exp,last_login
Albert,25,55000.5,true,2025-08-25,6,2025-08-25T15:30:00.000Z
Baskar,30,72000.0,false,2024-12-31,7,2024-12-31T08:45:15.000Z
Chetan,27,63000.75,true,2023-01-15,8,2023-01-15T20:00:00.000Z
Dravid,26,66000.5,true,2024-06-20,9,2024-05-22T15:40:55.000Z
Nishant,32,98700.0,false,2023-10-21,10,2020-10-31T06:45:45.000Z
David,29,34512.75,true,2023-03-15,4,2021-08-28T20:15:55.000Z
Mohan,33,34908.5,true,2022-04-15,12,2019-09-29T19:35:55.000Z
Niroop,35,49654.98,false,2021-02-18,3,2024-05-22T08:45:25.000Z
Pushpa,23,44111.99,true,2020-07-19,5,2023-09-25T20:00:00.000Z


##### Scenario 04
- date_format_map=**None**
- date_format_map = **{"date": "MM-dd-yyyy", "timestamp": "MM-dd-yyyy'T'HH:mm:ss.SSSXXX"}**

In [0]:
def cast_dataframe_columns_map(df, col_type_map, date_format_map=None):
    for col_name, data_type in col_type_map.items():
        data_type = data_type.lower()

        if data_type == "string":
            df = df.withColumn(col_name, F.col(col_name).cast("string"))

        elif data_type in ["int", "Int", "integer"]:
            df = df.withColumn(col_name, F.col(col_name).cast("int"))

        elif data_type in ["double", "float"]:
            df = df.withColumn(col_name, F.col(col_name).cast("double"))

        elif data_type == "boolean":
            df = df.withColumn(col_name, F.col(col_name).cast("boolean"))

        elif data_type == "date":
            df = df.withColumn(col_name, F.to_date(F.col(col_name), date_format_map.get("date")))

        elif data_type == "timestamp":
            df = df.withColumn(col_name, F.to_timestamp(F.col(col_name), date_format_map.get("timestamp")))
    return df

In [0]:
df_casted_cfrmt = cast_dataframe_columns_map(
    df_cust_format,
    col_type_map,
    {"date": "MM-dd-yyyy", "timestamp": "MM-dd-yyyy'T'HH:mm:ss.SSSXXX"}
)
display(df_casted_cfrmt)

name,age,salary,is_active,join_date,Exp,last_login
Albert,25,55000.5,true,2025-08-25,6,2025-08-25T15:30:00.000Z
Baskar,30,72000.0,false,2024-12-30,7,2024-12-31T08:45:15.000Z
Chetan,27,63000.75,true,2023-01-15,8,2023-01-15T20:00:00.000Z
Dravid,26,66000.5,true,2024-06-20,9,2024-05-22T15:40:55.000Z
Nishant,32,98700.0,false,2023-10-21,10,2020-10-31T06:45:45.000Z
David,29,34512.75,true,2023-03-15,4,2021-08-28T20:15:55.000Z
Mohan,33,34908.5,true,2022-04-15,12,2019-09-29T19:35:55.000Z
Niroop,35,49654.98,false,2021-02-18,3,2024-05-22T08:45:25.000Z
Pushpa,23,44111.99,true,2020-07-19,5,2023-09-25T20:00:00.000Z


##### Scenario 05
- date_format_map=**None**
- date_format_map = **{"my_date": "yyyy-MM-dd", "my_timestamp": "yyyy-MM-dd'T'HH:mm:ss.SSSXXX"}**

In [0]:
def cast_dataframe_columns_dtformat(df, col_type_map, date_format_map=None):
    for col_name, target_type in col_type_map.items():
        target_type = target_type.lower()
        fmt = None if date_format_map is None else date_format_map.get(col_name)

        if target_type == "string":
            df = df.withColumn(col_name, F.col(col_name).cast("string"))

        elif target_type in ["int", "integer"]:
            df = df.withColumn(col_name, F.col(col_name).cast("int"))

        elif target_type in ["double", "float"]:
            df = df.withColumn(col_name, F.col(col_name).cast("double"))

        elif target_type == "boolean":
            df = df.withColumn(col_name, F.col(col_name).cast("boolean"))

        elif target_type == "date":
            if fmt:
                df = df.withColumn(col_name, F.to_date(F.col(col_name), fmt))
            else:
                df = df.withColumn(col_name, F.to_date(F.col(col_name)))

        # For timestamp, apply both to_date and to_timestamp as new columns
        if target_type == "timestamp":
            if fmt:
                df = df.withColumn(f"{col_name}_date", F.to_date(F.col(col_name), fmt))
                df = df.withColumn(f"{col_name}_ts", F.to_timestamp(F.col(col_name), fmt))
            else:
                df = df.withColumn(f"{col_name}_date", F.to_date(F.col(col_name)))
                df = df.withColumn(f"{col_name}_ts", F.to_timestamp(F.col(col_name)))
    return df

In [0]:
date_format_map = {
    "my_date": "yyyy-MM-dd",
    "my_timestamp": "yyyy-MM-dd'T'HH:mm:ss.SSSXXX"
}

In [0]:
# Apply casting with correct timestamp format
df_casted_dtformat = cast_dataframe_columns_dtformat(df_default_format, col_type_map, date_format_map)
display(df_casted_dtformat)

name,age,salary,is_active,join_date,Exp,last_login,last_login_date,last_login_ts
Albert,25,55000.5,true,2025-08-25,6,2025-08-25T15:30:00.000+00:00,2025-08-25,2025-08-25T15:30:00.000Z
Baskar,30,72000.0,false,2024-12-31,7,2024-12-31T08:45:15.000+00:00,2024-12-31,2024-12-31T08:45:15.000Z
Chetan,27,63000.75,true,2023-01-15,8,2023-01-15T20:00:00.000+00:00,2023-01-15,2023-01-15T20:00:00.000Z
Dravid,26,66000.5,true,2024-06-20,9,2024-05-22T15:40:55.000+00:00,2024-05-22,2024-05-22T15:40:55.000Z
Nishant,32,98700.0,false,2023-10-21,10,2020-10-31T06:45:45.000+00:00,2020-10-31,2020-10-31T06:45:45.000Z
David,29,34512.75,true,2023-03-15,4,2021-08-28T20:15:55.000+00:00,2021-08-28,2021-08-28T20:15:55.000Z
Mohan,33,34908.5,true,2022-04-15,12,2019-09-29T19:35:55.000+00:00,2019-09-29,2019-09-29T19:35:55.000Z
Niroop,35,49654.98,false,2021-02-18,3,2024-05-22T08:45:25.000+00:00,2024-05-22,2024-05-22T08:45:25.000Z
Pushpa,23,44111.99,true,2020-07-19,5,2023-09-25T20:00:00.000+00:00,2023-09-25,2023-09-25T20:00:00.000Z


##### How Spark handles it?

If you call:

        F.to_date(F.col("start_date"))
- Spark uses its **default parser (yyyy-MM-dd)**.

If you call:

      F.to_date(F.col("start_date"), "yyyy-MM-dd'T'HH:mm:ss.SSSXXX")

- Spark will use your **custom format**.

##### 2) Why set None?

- **Without =None**, the argument becomes **mandatory**.

      def convert_dataframe_columns(df, col_type_map, date_format):
          ...


- Now you **must** always call:

      convert_dataframe_columns(df, {"start_date": "date"}, "yyyy-MM-dd'T'HH:mm:ss.SSSXXX")

- If you **don’t** pass a **format**, Python will raise:

      TypeError: missing 1 required positional argument: 'date_format'

- By making it **None**, you can handle **two cases**:
  - User **provided a format** → use it.
  - User **didn’t provide one** → either fall back to default Spark behavior or skip formatting.

##### Scenario 06
- date_format
- date_format = **"yyyy-MM-dd'T'HH:mm:ss.SSSXXX"**

In [0]:
def cast_dataframe_columns_none(df, col_type_map, date_format):
    for col_name, data_type in col_type_map.items():
        data_type = data_type.lower()

        if data_type == "string":
            df = df.withColumn(col_name, F.col(col_name).cast("string"))

        elif data_type in ["int", "integer"]:
            df = df.withColumn(col_name, F.col(col_name).cast("int"))

        elif data_type in ["double", "float"]:
            df = df.withColumn(col_name, F.col(col_name).cast("double"))

        elif data_type == "boolean":
            df = df.withColumn(col_name, F.col(col_name).cast("boolean"))

        elif data_type == "date":
            df = df.withColumn(col_name, F.to_date(F.col(col_name)))

        elif data_type == "timestamp":
            df = df.withColumn(col_name, F.to_timestamp(F.col(col_name)))
    return df

In [0]:
# Apply casting with correct timestamp format
df_casted_none = cast_dataframe_columns_none(df_default_format, col_type_map)
display(df_casted_none)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-2022302555744407>, line 2
      1 # Apply casting with correct timestamp format
----> 2 df_casted_none = cast_dataframe_columns_none(df_default_format, col_type_map)
      3 display(df_casted_none)

TypeError: cast_dataframe_columns_none() missing 1 required positional argument: 'date_format'

In [0]:
# Apply casting with correct timestamp format
df_casted_format_none = cast_dataframe_columns_none(df_default_format, col_type_map, "yyyy-MM-dd'T'HH:mm:ss.SSSXXX")
display(df_casted_format_none)

name,age,salary,is_active,join_date,Exp,last_login
Albert,25,55000.5,true,2025-08-25,6,2025-08-25T15:30:00.000Z
Baskar,30,72000.0,false,2024-12-31,7,2024-12-31T08:45:15.000Z
Chetan,27,63000.75,true,2023-01-15,8,2023-01-15T20:00:00.000Z
Dravid,26,66000.5,true,2024-06-20,9,2024-05-22T15:40:55.000Z
Nishant,32,98700.0,false,2023-10-21,10,2020-10-31T06:45:45.000Z
David,29,34512.75,true,2023-03-15,4,2021-08-28T20:15:55.000Z
Mohan,33,34908.5,true,2022-04-15,12,2019-09-29T19:35:55.000Z
Niroop,35,49654.98,false,2021-02-18,3,2024-05-22T08:45:25.000Z
Pushpa,23,44111.99,true,2020-07-19,5,2023-09-25T20:00:00.000Z


In [0]:
def cast_dataframe_columns_none_cust(df, col_type_map, date_format_map):
    for col_name, data_type in col_type_map.items():
        data_type = data_type.lower()

        if data_type == "string":
            df = df.withColumn(col_name, F.col(col_name).cast("string"))

        elif data_type in ["int", "integer"]:
            df = df.withColumn(col_name, F.col(col_name).cast("int"))

        elif data_type in ["double", "float"]:
            df = df.withColumn(col_name, F.col(col_name).cast("double"))

        elif data_type == "boolean":
            df = df.withColumn(col_name, F.col(col_name).cast("boolean"))

        elif data_type == "date":
            df = df.withColumn(col_name, F.to_date(F.col(col_name), date_format_map.get("date")))

        elif data_type == "timestamp":
            df = df.withColumn(col_name, F.to_timestamp(F.col(col_name), date_format_map.get("timestamp")))
    return df

In [0]:
df_casted_cfrmt_none = cast_dataframe_columns_none_cust(
    df_cust_format,
    col_type_map,
    {"date": "MM-dd-yyyy", "timestamp": "MM-dd-yyyy'T'HH:mm:ss.SSSXXX"}
)
display(df_casted_cfrmt_none)

name,age,salary,is_active,join_date,Exp,last_login
Albert,25,55000.5,true,2025-08-25,6,2025-08-25T15:30:00.000Z
Baskar,30,72000.0,false,2024-12-30,7,2024-12-31T08:45:15.000Z
Chetan,27,63000.75,true,2023-01-15,8,2023-01-15T20:00:00.000Z
Dravid,26,66000.5,true,2024-06-20,9,2024-05-22T15:40:55.000Z
Nishant,32,98700.0,false,2023-10-21,10,2020-10-31T06:45:45.000Z
David,29,34512.75,true,2023-03-15,4,2021-08-28T20:15:55.000Z
Mohan,33,34908.5,true,2022-04-15,12,2019-09-29T19:35:55.000Z
Niroop,35,49654.98,false,2021-02-18,3,2024-05-22T08:45:25.000Z
Pushpa,23,44111.99,true,2020-07-19,5,2023-09-25T20:00:00.000Z
